#  pyodbc

Sample code of pyodbc to connect with Microsoft SQL Server Database.

SQL Server Management:

https://docs.microsoft.com/en-us/sql/sql-server/?view=sql-server-ver15

SQL

https://www.sqlservertutorial.net/

In [29]:
# conda env py36db

In [1]:
# !pip install pyodbc

In [2]:
import pyodbc
import pandas as pd

In [3]:
# help(pyodbc)

In [4]:
#  Check drivers
for driver in pyodbc.drivers():
    print(driver)

SQL Server
SQL Server Native Client 11.0
ODBC Driver 17 for SQL Server
SQL Server Native Client RDA 11.0
ODBC Driver 13 for SQL Server
PostgreSQL ANSI(x64)
PostgreSQL Unicode(x64)
Amazon Redshift (x64)


### Connect to DB

In [5]:
#  SQL Server Authentication

server = 'DESKTOP-1GHH8VU\JSQL' 
database = 'testdb' 
username = 'sa' 
password = 'mssqlP@ssw0rd' 
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};'
                      'SERVER='+server+';'
                      'DATABASE='+database+';'
                      'UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

In [6]:
#  Window Authentication

server = 'DESKTOP-1GHH8VU\JSQL' 
database = 'testdb'
cnxn = pyodbc.connect('Driver={SQL Server};'
                      'SERVER='+server+';'
                      'DATABASE='+database+';'
                      'Trusted_Connection=yes;')

cursor = cnxn.cursor()

### Check version

In [7]:
cursor.execute("SELECT @@version;") 
row = cursor.fetchone() 
while row: 
    print(row[0])
    row = cursor.fetchone()

Microsoft SQL Server 2017 (RTM) - 14.0.1000.169 (X64) 
	Aug 22 2017 17:04:49 
	Copyright (C) 2017 Microsoft Corporation
	Developer Edition (64-bit) on Windows 10 Home 10.0 <X64> (Build 18362: )



###  Show Database

In [8]:
# SQL: 'EXEC sp_databases'

cursor = cnxn.cursor()
cursor.execute("SELECT name from sys.databases")
rows = cursor.fetchall()

for row in rows:
    sql = ("select type_desc, physical_name from %s.sys.database_files" % row.name)
    cursor.execute(sql)
    l1rows = cursor.fetchall()
    print("Database: " + row.name)
    for l1row in l1rows:
        print(l1row.type_desc + " " + l1row.physical_name)

Database: master
ROWS C:\Program Files\Microsoft SQL Server\MSSQL14.MSSQLSERVER2\MSSQL\DATA\master.mdf
LOG C:\Program Files\Microsoft SQL Server\MSSQL14.MSSQLSERVER2\MSSQL\DATA\mastlog.ldf
Database: tempdb
ROWS C:\Program Files\Microsoft SQL Server\MSSQL14.MSSQLSERVER2\MSSQL\DATA\tempdb.mdf
LOG C:\Program Files\Microsoft SQL Server\MSSQL14.MSSQLSERVER2\MSSQL\DATA\templog.ldf
ROWS C:\Program Files\Microsoft SQL Server\MSSQL14.MSSQLSERVER2\MSSQL\DATA\tempdb_mssql_2.ndf
ROWS C:\Program Files\Microsoft SQL Server\MSSQL14.MSSQLSERVER2\MSSQL\DATA\tempdb_mssql_3.ndf
ROWS C:\Program Files\Microsoft SQL Server\MSSQL14.MSSQLSERVER2\MSSQL\DATA\tempdb_mssql_4.ndf
ROWS C:\Program Files\Microsoft SQL Server\MSSQL14.MSSQLSERVER2\MSSQL\DATA\tempdb_mssql_5.ndf
ROWS C:\Program Files\Microsoft SQL Server\MSSQL14.MSSQLSERVER2\MSSQL\DATA\tempdb_mssql_6.ndf
ROWS C:\Program Files\Microsoft SQL Server\MSSQL14.MSSQLSERVER2\MSSQL\DATA\tempdb_mssql_7.ndf
ROWS C:\Program Files\Microsoft SQL Server\MSSQL14.MSSQLSE

###  Create Table

In [9]:
cursor.execute('''

               IF NOT EXISTS (SELECT * FROM sysobjects WHERE name='customer' and xtype='U')
               CREATE TABLE customer
               (
               Name nvarchar(50),
               Age int,
               City nvarchar(50),
               Date date
               )

               ''')

cnxn.commit()

###  Drop Table

In [10]:
# cursor.execute("DROP TABLE customer")

###  Show all tables
columns:
1. table_cat: The catalog name.
2. table_schem: The schema name.
3. table_name: The table name.
4. table_type

Schema name

- dbo
- sys
- INFORMATION_SCHEMA

In [11]:
# cursor = cnxn.cursor()                                                                                                                                  
# for row in cursor.tables():
#     print(row)

In [12]:
table_info = cursor.tables()
table_info = table_info.fetchall()
df = pd.DataFrame(table_info, columns=['table_info'])

# Split column
df = df['table_info'].astype(str).str.split(expand=True)

# All column
df.columns =['table_cat','table_schem','table_name','table_type','memo']

#  Replace symbol
df = df.replace("'", "", regex=True)
df = df.replace("\(", "", regex=True)
df = df.replace("\)", "", regex=True)
df = df.replace(",", "", regex=True)

#  Filter only dbo
df = df[df['table_schem'].astype(str).str.contains('dbo', na=False)]

df

,table_cat,table_schem,table_name,table_type,memo
0,testdb,dbo,customer,TABLE,None


###  Show table meta data
Columns:
1. table_cat
2. table_schem
3. table_name
4. column_name
5. data_type
6. type_name
7. column_size
8. buffer_length
9. decimal_digits
10. num_prec_radix
11. nullable
12. remarks
13. column_def
14. sql_data_type
15. sql_datetime_sub
16. char_octet_length
17. ordinal_position

In [13]:
meta = cursor.columns(table='customer')                                                                                                              

for row in meta:
    print(row)

('testdb', 'dbo', 'customer', 'Name', -9, 'nvarchar', 50, 100, None, None, 1, None, None, -9, None, 100, 1, 'YES', 39)
('testdb', 'dbo', 'customer', 'Age', 4, 'int', 10, 4, 0, 10, 1, None, None, 4, None, None, 2, 'YES', 38)
('testdb', 'dbo', 'customer', 'City', -9, 'nvarchar', 50, 100, None, None, 1, None, None, -9, None, 100, 3, 'YES', 39)
('testdb', 'dbo', 'customer', 'Date', -9, 'date', 10, 20, None, None, 1, None, None, -9, None, None, 4, 'YES', 0)


In [14]:
meta = cursor.columns(table='customer')                                                                                                              

for row in meta:
    print(row.column_name, row.type_name)

Name nvarchar
Age int
City nvarchar
Date date


### Insert Records

In [15]:
cursor.execute('''

                INSERT INTO testdb.dbo.customer (Name, Age, City, Date)
                VALUES
                ('Sakura',20,'Tokyo', '2020-01-01'),
                ('Neo',30,'London', '2020-04-15'),
                ('Patrick',40,'', '2019-10-21')  

                ''')
print(cursor.rowcount, 'records inserted')

cnxn.commit()

3 records inserted


In [16]:
cursor.execute("INSERT INTO customer (Name, Age, City, Date) values (?, ?, ?, ?)",
               'Tom', '50','California', '2020-02-03')
print(cursor.rowcount, 'records inserted')

cnxn.commit()

1 records inserted


###  Update Records

In [17]:
cursor.execute('''
                UPDATE TestDB.dbo.customer
                SET Age = 50, City = 'Bangkok'
                WHERE Name = 'Patrick'
                ''')
print(cursor.rowcount, 'records updated')

cnxn.commit()

16 records updated


###  Delete  Records

In [18]:
cursor.execute('''
                DELETE FROM TestDB.dbo.customer
                WHERE Name = 'Tom'
                ''')
print(cursor.rowcount, 'records deleted')

cnxn.commit()

1 records deleted


###  Query Data

In [19]:
#  SELECT ALL

cursor.execute("SELECT * FROM customer")
rows = cursor.fetchall()

for row in rows:
    print(row, end='\n')

('Sakura', 20, 'Tokyo', '2020-01-01')
('Neo', 30, 'London', '2020-04-15')
('Patrick', 50, 'Bangkok', '2019-10-21')
('Sakura', 20, 'Tokyo', '2020-01-01')
('Sakura', 20, 'Tokyo', '2020-01-01')
('Neo', 30, 'London', '2020-04-15')
('Patrick', 50, 'Bangkok', '2019-10-21')
('Neo', 30, 'London', '2020-04-15')
('Patrick', 50, 'Bangkok', '2019-10-21')
('Sakura', 20, 'Tokyo', '2020-01-01')
('Neo', 30, 'London', '2020-04-15')
('Patrick', 50, 'Bangkok', '2019-10-21')
('Sakura', 20, 'Tokyo', '2020-01-01')
('Sakura', 20, 'Tokyo', '2020-01-01')
('Neo', 30, 'London', '2020-04-15')
('Patrick', 50, 'Bangkok', '2019-10-21')
('Neo', 30, 'London', '2020-04-15')
('Patrick', 50, 'Bangkok', '2019-10-21')
('Sakura', 20, 'Tokyo', '2020-01-01')
('Neo', 30, 'London', '2020-04-15')
('Patrick', 50, 'Bangkok', '2019-10-21')
('Sakura', 20, 'Tokyo', '2020-01-01')
('Sakura', 20, 'Tokyo', '2020-01-01')
('Neo', 30, 'London', '2020-04-15')
('Patrick', 50, 'Bangkok', '2019-10-21')
('Neo', 30, 'London', '2020-04-15')
('Patr

In [20]:
#  WHERE text

cursor.execute("SELECT * FROM customer WHERE Name='Patrick'")
rows = cursor.fetchall()

for row in rows:
    print(row, end='\n')

('Patrick', 50, 'Bangkok', '2019-10-21')
('Patrick', 50, 'Bangkok', '2019-10-21')
('Patrick', 50, 'Bangkok', '2019-10-21')
('Patrick', 50, 'Bangkok', '2019-10-21')
('Patrick', 50, 'Bangkok', '2019-10-21')
('Patrick', 50, 'Bangkok', '2019-10-21')
('Patrick', 50, 'Bangkok', '2019-10-21')
('Patrick', 50, 'Bangkok', '2019-10-21')
('Patrick', 50, 'Bangkok', '2019-10-21')
('Patrick', 50, 'Bangkok', '2019-10-21')
('Patrick', 50, 'Bangkok', '2019-10-21')
('Patrick', 50, 'Bangkok', '2019-10-21')
('Patrick', 50, 'Bangkok', '2019-10-21')
('Patrick', 50, 'Bangkok', '2019-10-21')
('Patrick', 50, 'Bangkok', '2019-10-21')
('Patrick', 50, 'Bangkok', '2019-10-21')


In [21]:
#  WHERE int

cursor.execute("SELECT * FROM customer WHERE Age > 30")
rows = cursor.fetchall()

for row in rows:
    print(row, end='\n')

('Patrick', 50, 'Bangkok', '2019-10-21')
('Patrick', 50, 'Bangkok', '2019-10-21')
('Patrick', 50, 'Bangkok', '2019-10-21')
('Patrick', 50, 'Bangkok', '2019-10-21')
('Patrick', 50, 'Bangkok', '2019-10-21')
('Patrick', 50, 'Bangkok', '2019-10-21')
('Patrick', 50, 'Bangkok', '2019-10-21')
('Patrick', 50, 'Bangkok', '2019-10-21')
('Patrick', 50, 'Bangkok', '2019-10-21')
('Patrick', 50, 'Bangkok', '2019-10-21')
('Patrick', 50, 'Bangkok', '2019-10-21')
('Patrick', 50, 'Bangkok', '2019-10-21')
('Patrick', 50, 'Bangkok', '2019-10-21')
('Patrick', 50, 'Bangkok', '2019-10-21')
('Patrick', 50, 'Bangkok', '2019-10-21')
('Patrick', 50, 'Bangkok', '2019-10-21')


In [22]:
#  WHERE date

cursor.execute("SELECT * FROM customer WHERE Date > '2019-12-31'")
rows = cursor.fetchall()

for row in rows:
    print(row, end='\n')

('Sakura', 20, 'Tokyo', '2020-01-01')
('Neo', 30, 'London', '2020-04-15')
('Sakura', 20, 'Tokyo', '2020-01-01')
('Sakura', 20, 'Tokyo', '2020-01-01')
('Neo', 30, 'London', '2020-04-15')
('Neo', 30, 'London', '2020-04-15')
('Sakura', 20, 'Tokyo', '2020-01-01')
('Neo', 30, 'London', '2020-04-15')
('Sakura', 20, 'Tokyo', '2020-01-01')
('Sakura', 20, 'Tokyo', '2020-01-01')
('Neo', 30, 'London', '2020-04-15')
('Neo', 30, 'London', '2020-04-15')
('Sakura', 20, 'Tokyo', '2020-01-01')
('Neo', 30, 'London', '2020-04-15')
('Sakura', 20, 'Tokyo', '2020-01-01')
('Sakura', 20, 'Tokyo', '2020-01-01')
('Neo', 30, 'London', '2020-04-15')
('Neo', 30, 'London', '2020-04-15')
('Sakura', 20, 'Tokyo', '2020-01-01')
('Neo', 30, 'London', '2020-04-15')
('Sakura', 20, 'Tokyo', '2020-01-01')
('Sakura', 20, 'Tokyo', '2020-01-01')
('Neo', 30, 'London', '2020-04-15')
('Neo', 30, 'London', '2020-04-15')
('Sakura', 20, 'Tokyo', '2020-01-01')
('Neo', 30, 'London', '2020-04-15')
('Sakura', 20, 'Tokyo', '2020-01-01')


###  Create dataframe

In [23]:
import pandas as pd

query = "SELECT * FROM testdb.dbo.customer"

df = pd.read_sql(query, cnxn)
df

,Name,Age,City,Date
0,Sakura,20,Tokyo,2020-01-01
1,Neo,30,London,2020-04-15
2,Patrick,50,Bangkok,2019-10-21
3,Sakura,20,Tokyo,2020-01-01
4,Sakura,20,Tokyo,2020-01-01
5,Neo,30,London,2020-04-15
6,Patrick,50,Bangkok,2019-10-21
7,Neo,30,London,2020-04-15
8,Patrick,50,Bangkok,2019-10-21
9,Sakura,20,Tokyo,2020-01-01


###  Insert dataframe to SQL Server

In [24]:
# Insert Dataframe into SQL Server:

totalrow = 0
for index, row in df.iterrows():
    cursor.execute("INSERT INTO dbo.customer (Name,Age,City,Date) values(?,?,?,?)", 
                   row.Name, row.Age, row.City, row.Date)
    totalrow +=cursor.rowcount

print(totalrow, 'records inserted')

cnxn.commit()

48 records inserted


## Functions

In [30]:
import pyodbc


def dblogin():
    server = 'DESKTOP-1GHH8VU\JSQL' 
    database = 'testdb' 
    username = 'sa' 
    password = 'mssqlP@ssw0rd' 

    cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};'
                      'SERVER='+server+';'
                      'DATABASE='+database+';'
                      'UID='+username+';PWD='+ password)
    return cnxn.cursor()


def select_execute(con, slq):
        cursor.execute(slq)
        rows = cursor.fetchall()
        return rows


if __name__ == '__main__':
    cursor = dblogin()
    sql =  '''SELECT *
                FROM customer'''
    rows = select_execute(cursor, sql)
    for row in rows:
        print(row, end='\n')

('Sakura', 20, 'Tokyo', datetime.date(2020, 1, 1))
('Neo', 30, 'London', datetime.date(2020, 4, 15))
('Patrick', 50, 'Bangkok', datetime.date(2019, 10, 21))
('Sakura', 20, 'Tokyo', datetime.date(2020, 1, 1))
('Sakura', 20, 'Tokyo', datetime.date(2020, 1, 1))
('Neo', 30, 'London', datetime.date(2020, 4, 15))
('Patrick', 50, 'Bangkok', datetime.date(2019, 10, 21))
('Neo', 30, 'London', datetime.date(2020, 4, 15))
('Patrick', 50, 'Bangkok', datetime.date(2019, 10, 21))
('Sakura', 20, 'Tokyo', datetime.date(2020, 1, 1))
('Neo', 30, 'London', datetime.date(2020, 4, 15))
('Patrick', 50, 'Bangkok', datetime.date(2019, 10, 21))
('Sakura', 20, 'Tokyo', datetime.date(2020, 1, 1))
('Sakura', 20, 'Tokyo', datetime.date(2020, 1, 1))
('Neo', 30, 'London', datetime.date(2020, 4, 15))
('Patrick', 50, 'Bangkok', datetime.date(2019, 10, 21))
('Neo', 30, 'London', datetime.date(2020, 4, 15))
('Patrick', 50, 'Bangkok', datetime.date(2019, 10, 21))
('Sakura', 20, 'Tokyo', datetime.date(2020, 1, 1))
('Neo',

###  Close connection and remove the cursor

In [28]:
cursor.close()
cnxn.close()